# GPT-2 Model

We are using HuggingFace's model!

In [ ]:
# uncomment for colab
%pip install transformers datasets accelerate nvidia-ml-py3 evaluate

# import hugging face
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# for colab to keep track of utilization

from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

# print GPU utilization
print_gpu_utilization()

In [ ]:
# create smaller dataset from our subset data
from datasets import Dataset
import pandas as pd
filename = 'data/5000_booksummaries.zip'
tokens_df = pd.read_csv(filename)
tokens_df.head(5)

In [ ]:
# split data into train and test/eval data
from sklearn.model_selection import train_test_split

# split into train (80%), val (10%), test (10%)
train_data, test_eval_dataset = train_test_split(tokens_df, test_size=0.2, random_state=8)
eval_set, test_set = train_test_split(test_eval_dataset, test_size=0.5, random_state=8)

# create HuggingFace Datasets
train_ds = Dataset.from_pandas(train_data)
eval_ds = Dataset.from_pandas(eval_set)
test_ds = Dataset.from_pandas(test_set)

In [ ]:
# tokenize datasets
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["Text"], truncation=True)

train_tok_ds = train_ds.map(tokenize_function, batched=True).shuffle(seed=42)
eval_tok_ds = eval_ds.map(tokenize_function, batched=True).shuffle(seed=42)

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained('gpt2')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

epochs = 3 # change between 3 and 5

training_args = TrainingArguments(
        output_dir="temp_model",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        evaluation_strategy='epoch',
        per_device_train_batch_size=4,
        num_train_epochs=epochs,   
        save_total_limit=1,
        logging_steps=250,
        gradient_accumulation_steps=4, 
        gradient_checkpointing=True, 
        fp16=True
) 

trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_tok_ds,
        eval_dataset=eval_tok_ds,
    )

In [ ]:
trainer.train()

In [ ]:
# save local version
checkpoint = "./model_config"
model.save_pretrained(checkpoint)
tokenizer.save_pretrained(checkpoint)

In [ ]:
%zip -r /content/model_setup_5000_3.zip /content/model_config

# Testing the Model

Now, we can utilize our trained model parameters to conduct analysis and test on our dataset!

In [ ]:
# load into model and tokenizer
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)

In [ ]:
# load input prompt
input_prompt = "Generate a book summary with genre science fiction:\n"
inputs = tokenizer(input_prompt, return_tensors="pt")

# generate output from pretrained experiments (see baseline file)
outputs = model.generate(**inputs, 
    max_length=150, 
    num_beams=2, 
    no_repeat_ngram_size=2, 
    do_sample=True,
    early_stopping=True)

# decode output and print out summary
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0])